# 3. Feature Engineering

In [1]:
import pickle
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
import numpy as np

In [2]:
path_df = "Data/News_dataset.pickle"

with open(path_df, 'rb') as data:
    df = pickle.load(data)

In [3]:
df.head()

,File_Name,Category,Content,id,News_length
0,Data/bbc\business\001.txt,business,Ad sales boost Time Warner profit\n\nQuarterly...,1,2559
1,Data/bbc\business\002.txt,business,Dollar gains on Greenspan speech\n\nThe dollar...,1,2252
2,Data/bbc\business\003.txt,business,Yukos unit buyer faces loan claim\n\nThe owner...,1,1551
3,Data/bbc\business\004.txt,business,High fuel prices hit BA's profits\n\nBritish A...,1,2401
4,Data/bbc\business\005.txt,business,Pernod takeover talk lifts Domecq\n\nShares in...,1,1569


In [4]:
df.loc[1]['Content']

'Dollar gains on Greenspan speech\n\nThe dollar has hit its highest level against the euro in almost three months after the Federal Reserve head said the US trade deficit is set to stabilise.\n\nAnd Alan Greenspan highlighted the US government\'s willingness to curb spending and rising household savings as factors which may help to reduce it. In late trading in New York, the dollar reached $1.2871 against the euro, from $1.2974 on Thursday. Market concerns about the deficit has hit the greenback in recent months. On Friday, Federal Reserve chairman Mr Greenspan\'s speech in London ahead of the meeting of G7 finance ministers sent the dollar higher after it had earlier tumbled on the back of worse-than-expected US jobs data. "I think the chairman\'s taking a much more sanguine view on the current account deficit than he\'s taken for some time," said Robert Sinche, head of currency strategy at Bank of America in New York. "He\'s taking a longer-term view, laying out a set of conditions u

## 1. Text Cleaning and Preparation

Berikut adalah langkah-langkah text cleaning dan preprocessing:
1. Hapus karakter khusus '\r', '\n', '"',
2. Ubah ke bentuk lowercase
3. Hapus sejumlah simbol
4. Hapus 's, misal student's name
5. Lematisasi, mengubah ke bentuk kata dasar dengan bantuan wordnet
6. Hapus stopwords

In [5]:
df['Content_Parsed_1'] = df['Content'].str.replace("\r", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("\n", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("    ", " ")

In [6]:
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace('"', '')

In [7]:
df['Content_Parsed_2'] = df['Content_Parsed_1'].str.lower()

In [8]:

punctuation_signs = list("?:!.,;")
df['Content_Parsed_3'] = df['Content_Parsed_2']

for punct_sign in punctuation_signs:
    df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, '')

In [9]:
# Remove possessive pronouns
df['Content_Parsed_4'] = df['Content_Parsed_3'].str.replace("'s", "")

In [10]:
# Downloading punkt and wordnet from NLTK
nltk.download('punkt')
print("------------------------------------------------------------")
nltk.download('wordnet')

------------------------------------------------------------


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASLAB\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ASLAB\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [11]:
# Saving the lemmatizer into an object
wordnet_lemmatizer = WordNetLemmatizer()

In [12]:
nrows = len(df)
lemmatized_text_list = []

for row in range(0, nrows):
    
    # Create an empty list containing lemmatized words
    lemmatized_list = []
    
    # Save the text and its words into an object
    text = df.loc[row]['Content_Parsed_4']
    text_words = text.split(" ")

    # Iterate through every word to lemmatize
    for word in text_words:
        lemmatized_list.append(wordnet_lemmatizer.lemmatize(word, pos="v"))
        
    # Join the list
    lemmatized_text = " ".join(lemmatized_list)
    
    # Append to the list containing the texts
    lemmatized_text_list.append(lemmatized_text)

In [13]:
df['Content_Parsed_5'] = lemmatized_text_list

In [14]:
# Downloading the stop words list
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASLAB\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
# Loading the stop words in english
stop_words = list(stopwords.words('english'))

In [16]:
stop_words[0:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [17]:
df['Content_Parsed_6'] = df['Content_Parsed_5']

for stop_word in stop_words:

    regex_stopword = r"\b" + stop_word + r"\b"
    df['Content_Parsed_6'] = df['Content_Parsed_6'].str.replace(regex_stopword, '')

In [18]:
df.loc[5]['Content']

'Japan narrowly escapes recession\n\nJapan\'s economy teetered on the brink of a technical recession in the three months to September, figures show.\n\nRevised figures indicated growth of just 0.1% - and a similar-sized contraction in the previous quarter. On an annual basis, the data suggests annual growth of just 0.2%, suggesting a much more hesitant recovery than had previously been thought. A common technical definition of a recession is two successive quarters of negative growth.\n\nThe government was keen to play down the worrying implications of the data. "I maintain the view that Japan\'s economy remains in a minor adjustment phase in an upward climb, and we will monitor developments carefully," said economy minister Heizo Takenaka. But in the face of the strengthening yen making exports less competitive and indications of weakening economic conditions ahead, observers were less sanguine. "It\'s painting a picture of a recovery... much patchier than previously thought," said Pa

In [19]:
df.loc[5]['Content_Parsed_1']

"Japan narrowly escapes recession  Japan's economy teetered on the brink of a technical recession in the three months to September, figures show.  Revised figures indicated growth of just 0.1% - and a similar-sized contraction in the previous quarter. On an annual basis, the data suggests annual growth of just 0.2%, suggesting a much more hesitant recovery than had previously been thought. A common technical definition of a recession is two successive quarters of negative growth.  The government was keen to play down the worrying implications of the data. I maintain the view that Japan's economy remains in a minor adjustment phase in an upward climb, and we will monitor developments carefully, said economy minister Heizo Takenaka. But in the face of the strengthening yen making exports less competitive and indications of weakening economic conditions ahead, observers were less sanguine. It's painting a picture of a recovery... much patchier than previously thought, said Paul Sheard, ec

In [20]:
df.loc[5]['Content_Parsed_2']

"japan narrowly escapes recession  japan's economy teetered on the brink of a technical recession in the three months to september, figures show.  revised figures indicated growth of just 0.1% - and a similar-sized contraction in the previous quarter. on an annual basis, the data suggests annual growth of just 0.2%, suggesting a much more hesitant recovery than had previously been thought. a common technical definition of a recession is two successive quarters of negative growth.  the government was keen to play down the worrying implications of the data. i maintain the view that japan's economy remains in a minor adjustment phase in an upward climb, and we will monitor developments carefully, said economy minister heizo takenaka. but in the face of the strengthening yen making exports less competitive and indications of weakening economic conditions ahead, observers were less sanguine. it's painting a picture of a recovery... much patchier than previously thought, said paul sheard, ec

In [21]:
df.loc[5]['Content_Parsed_3']

"japan narrowly escapes recession  japan's economy teetered on the brink of a technical recession in the three months to september figures show  revised figures indicated growth of just 01% - and a similar-sized contraction in the previous quarter on an annual basis the data suggests annual growth of just 02% suggesting a much more hesitant recovery than had previously been thought a common technical definition of a recession is two successive quarters of negative growth  the government was keen to play down the worrying implications of the data i maintain the view that japan's economy remains in a minor adjustment phase in an upward climb and we will monitor developments carefully said economy minister heizo takenaka but in the face of the strengthening yen making exports less competitive and indications of weakening economic conditions ahead observers were less sanguine it's painting a picture of a recovery much patchier than previously thought said paul sheard economist at lehman br

In [22]:
df.loc[5]['Content_Parsed_4']

'japan narrowly escapes recession  japan economy teetered on the brink of a technical recession in the three months to september figures show  revised figures indicated growth of just 01% - and a similar-sized contraction in the previous quarter on an annual basis the data suggests annual growth of just 02% suggesting a much more hesitant recovery than had previously been thought a common technical definition of a recession is two successive quarters of negative growth  the government was keen to play down the worrying implications of the data i maintain the view that japan economy remains in a minor adjustment phase in an upward climb and we will monitor developments carefully said economy minister heizo takenaka but in the face of the strengthening yen making exports less competitive and indications of weakening economic conditions ahead observers were less sanguine it painting a picture of a recovery much patchier than previously thought said paul sheard economist at lehman brothers

In [23]:
df.loc[5]['Content_Parsed_5']

'japan narrowly escape recession  japan economy teeter on the brink of a technical recession in the three months to september figure show  revise figure indicate growth of just 01% - and a similar-sized contraction in the previous quarter on an annual basis the data suggest annual growth of just 02% suggest a much more hesitant recovery than have previously be think a common technical definition of a recession be two successive quarter of negative growth  the government be keen to play down the worry implications of the data i maintain the view that japan economy remain in a minor adjustment phase in an upward climb and we will monitor developments carefully say economy minister heizo takenaka but in the face of the strengthen yen make export less competitive and indications of weaken economic condition ahead observers be less sanguine it paint a picture of a recovery much patchier than previously think say paul sheard economist at lehman brothers in tokyo improvements in the job marke

In [24]:
df.loc[5]['Content_Parsed_6']

'japan narrowly escape recession  japan economy teeter   brink   technical recession   three months  september figure show  revise figure indicate growth   01% -   similar-sized contraction   previous quarter   annual basis  data suggest annual growth   02% suggest  much  hesitant recovery   previously  think  common technical definition   recession  two successive quarter  negative growth   government  keen  play   worry implications   data  maintain  view  japan economy remain   minor adjustment phase   upward climb    monitor developments carefully say economy minister heizo takenaka    face   strengthen yen make export less competitive  indications  weaken economic condition ahead observers  less sanguine  paint  picture   recovery much patchier  previously think say paul sheard economist  lehman brothers  tokyo improvements   job market apparently  yet  fee   domestic demand  private consumption   02%   third quarter '

In [25]:
df.head(1)

,File_Name,Category,Content,id,News_length,Content_Parsed_1,Content_Parsed_2,Content_Parsed_3,Content_Parsed_4,Content_Parsed_5,Content_Parsed_6
0,Data/bbc\business\001.txt,business,Ad sales boost Time Warner profit\n\nQuarterly...,1,2559,Ad sales boost Time Warner profit Quarterly p...,ad sales boost time warner profit quarterly p...,ad sales boost time warner profit quarterly p...,ad sales boost time warner profit quarterly p...,ad sales boost time warner profit quarterly p...,ad sales boost time warner profit quarterly p...


In [26]:
list_columns = ["File_Name", "Category", "Content", "Content_Parsed_6"]
df = df[list_columns]

df = df.rename(columns={'Content_Parsed_6': 'Content_Parsed'})

In [27]:
df.head()

,File_Name,Category,Content,Content_Parsed
0,Data/bbc\business\001.txt,business,Ad sales boost Time Warner profit\n\nQuarterly...,ad sales boost time warner profit quarterly p...
1,Data/bbc\business\002.txt,business,Dollar gains on Greenspan speech\n\nThe dollar...,dollar gain greenspan speech dollar hit h...
2,Data/bbc\business\003.txt,business,Yukos unit buyer faces loan claim\n\nThe owner...,yukos unit buyer face loan claim owners emb...
3,Data/bbc\business\004.txt,business,High fuel prices hit BA's profits\n\nBritish A...,high fuel price hit ba profit british airways...
4,Data/bbc\business\005.txt,business,Pernod takeover talk lifts Domecq\n\nShares in...,pernod takeover talk lift domecq share uk dr...


## 2. Label Coding

Mengubah nama kategori ke bentuk angka

In [28]:
category_codes = {
    'business': 0,
    'entertainment': 1,
    'politics': 2,
    'sport': 3,
    'tech': 4
}

In [29]:
# Category mapping
df['Category_Code'] = df['Category']
df = df.replace({'Category_Code':category_codes})

In [30]:
df.head()

,File_Name,Category,Content,Content_Parsed,Category_Code
0,Data/bbc\business\001.txt,business,Ad sales boost Time Warner profit\n\nQuarterly...,ad sales boost time warner profit quarterly p...,0
1,Data/bbc\business\002.txt,business,Dollar gains on Greenspan speech\n\nThe dollar...,dollar gain greenspan speech dollar hit h...,0
2,Data/bbc\business\003.txt,business,Yukos unit buyer faces loan claim\n\nThe owner...,yukos unit buyer face loan claim owners emb...,0
3,Data/bbc\business\004.txt,business,High fuel prices hit BA's profits\n\nBritish A...,high fuel price hit ba profit british airways...,0
4,Data/bbc\business\005.txt,business,Pernod takeover talk lifts Domecq\n\nShares in...,pernod takeover talk lift domecq share uk dr...,0


## 3. Train-Test Split

Dari total data yang ada, kita ingin pisahkan ke dalam training data dan testing data.

In [31]:
X_train, X_test, y_train, y_test = train_test_split(df['Content_Parsed'], 
                                                    df['Category_Code'], 
                                                    test_size=0.15, 
                                                    random_state=8)

In [32]:
print(len(df))
print(len(X_train))
print(len(X_test))

2225
1891
334


## 4. Text Representation

In [33]:
# Parameter election
ngram_range = (1,2)
min_df = 10
max_df = 1.
max_features = None

In [34]:

tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)
                        
features_train = tfidf.fit_transform(X_train).toarray()
labels_train = y_train
print(features_train.shape)

features_test = tfidf.transform(X_test).toarray()
labels_test = y_test
print(features_test.shape)

(1891, 5854)
(334, 5854)


In [35]:
from sklearn.feature_selection import chi2
import numpy as np

for Product, category_id in sorted(category_codes.items()):
    features_chi2 = chi2(features_train, labels_train == category_id)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf.get_feature_names())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
    print("# '{}' category:".format(Product))
    print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-5:])))
    print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-2:])))
    print("")

# 'business' category:
  . Most correlated unigrams:
. stock
. economy
. oil
. growth
. bank
  . Most correlated bigrams:
. analysts say
. stock market

# 'entertainment' category:
  . Most correlated unigrams:
. singer
. actor
. star
. award
. film
  . Most correlated bigrams:
. win best
. box office

# 'politics' category:
  . Most correlated unigrams:
. tory
. party
. blair
. election
. labour
  . Most correlated bigrams:
. tony blair
. mr blair

# 'sport' category:
  . Most correlated unigrams:
. injury
. champion
. cup
. coach
. match
  . Most correlated bigrams:
. grand slam
. six nations

# 'tech' category:
  . Most correlated unigrams:
. digital
. technology
. computer
. software
. users
  . Most correlated bigrams:
. let people
. mobile phone



In [36]:
category_codes.items()

dict_items([('business', 0), ('entertainment', 1), ('politics', 2), ('sport', 3), ('tech', 4)])

In [37]:
bigrams

['say move',
 'two days',
 'june 2004',
 'press report',
 'years ago',
 'say would',
 'give chance',
 'joint venture',
 'would go',
 'hop would',
 'try get',
 'come months',
 'three four',
 'five years',
 'ago say',
 'say really',
 'prisoner azkaban',
 'potter prisoner',
 'film say',
 'two firm',
 'go say',
 'taylor say',
 'release us',
 'take action',
 'id card',
 'next five',
 'months ago',
 'say problem',
 'director general',
 'good enough',
 'get way',
 'much money',
 'third quarter',
 'years say',
 'would look',
 'one say',
 'say john',
 'president george',
 'plan include',
 'people come',
 'record label',
 'spokesman tell',
 'czech republic',
 'time come',
 'say one',
 'high level',
 'last september',
 'second place',
 'voice concern',
 'however mr',
 'four months',
 'say work',
 'year best',
 'say start',
 'years later',
 'film release',
 'make new',
 'first three',
 'earlier year',
 'go back',
 'say say',
 'five time',
 'make difference',
 'get grip',
 'chapter 11',
 'group say

In [38]:
# X_train
with open('Data/X_train32c.pickle', 'wb') as output:
    pickle.dump(X_train, output)
    
# X_test    
with open('Data/X_test32c.pickle', 'wb') as output:
    pickle.dump(X_test, output)
    
# y_train
with open('Data/y_train32c.pickle', 'wb') as output:
    pickle.dump(y_train, output)
    
# y_test
with open('Data/y_test32c.pickle', 'wb') as output:
    pickle.dump(y_test, output)
    
# df
with open('Data/df32c.pickle', 'wb') as output:
    pickle.dump(df, output)
    
# features_train
with open('Data/features_train32c.pickle', 'wb') as output:
    pickle.dump(features_train, output)

# labels_train
with open('Data/labels_train32c.pickle', 'wb') as output:
    pickle.dump(labels_train, output)

# features_test
with open('Data/features_test32c.pickle', 'wb') as output:
    pickle.dump(features_test, output)

# labels_test
with open('Data/labels_test32c.pickle', 'wb') as output:
    pickle.dump(labels_test, output)
    
# TF-IDF object
with open('Data/tfidf32c.pickle', 'wb') as output:
    pickle.dump(tfidf, output)